### Dataset
[Copernicus ER5 land](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-land?tab=overview)
### Connexion à l'api 
1. Créer un compte Copernicus : https://cds.climate.copernicus.eu/user/register?destination=%2F%23!%2Fhome
2. Suivre le guide d'installation (https://cds.climate.copernicus.eu/api-how-to) pour créer un fichier .cdsapirc contenant les informations de connexion.
3. Connexion à l'aide de la librairie [cdsapi](https://pypi.org/project/cdsapi/)
### Acceptation des conditions d'utilisation
Si cela n'a pas été depuis le site, à la première exécution, suivre le lien renvoyé par renvoyé par le connecteur pour accepter les conditions d'utilisation du dataset.

In [4]:
import cdsapi
import netCDF4 as nc
import numpy as np
import datetime as dt
import pandas as pd
import os

In [5]:
DATE_START = dt.datetime(1900,1,1)

In [15]:
request = {
        "variable": [
            "evaporation_from_bare_soil",
            "evaporation_from_open_water_surfaces_excluding_oceans",
            "evaporation_from_the_top_of_canopy",
            "evaporation_from_vegetation_transpiration",
            "potential_evaporation",
            "total_evaporation",
            "total_precipitation",
        ],
        "year": "2022",
        "month": "01",
        "day": [
            "01",
            "02",
            "03",
            "04",
            "05",
            "06",
            "07",
            "08",
            "09",
            "10",
            "11",
            "12",
            "13",
            "14",
            "15",
            "16",
            "17",
            "18",
            "19",
            "20",
            "21",
            "22",
            "23",
            "24",
            "25",
            "26",
            "27",
            "28",
            "29",
            "30",
            "31",
        ],
        "time": "12:00",
        "area": [
            52, -5, 42,
                10,
        ],
        "format": "netcdf",
    }
target = "../data/download.nc"
name = "reanalysis-era5-land" 

In [18]:
c = cdsapi.Client()
if not os.path.isfile(target):
    c.retrieve(
        name=name,
        request=request,
        target=target,
    )

2023-02-25 11:26:15,636 INFO Welcome to the CDS
2023-02-25 11:26:15,637 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-land
2023-02-25 11:26:15,722 INFO Request is completed
2023-02-25 11:26:15,723 INFO Downloading https://download-0000-clone.copernicus-climate.eu/cache-compute-0000/cache/data2/adaptor.mars.internal-1677320621.3808982-8170-6-3342fd14-f7e3-4f19-87d2-ec665fa88d8d.nc to ../data/download.nc (6.3M)
2023-02-25 11:26:17,403 INFO Download rate 3.8M/s   


In [20]:
nc_file = nc.Dataset(target)

In [21]:
nc_file.variables.keys()

dict_keys(['longitude', 'latitude', 'time', 'evabs', 'evaow', 'evatc', 'evavt', 'pev', 'e', 'tp'])

In [22]:
nc_file.variables["time"]

<class 'netCDF4._netCDF4.Variable'>
int32 time(time)
    units: hours since 1900-01-01 00:00:00.0
    long_name: time
    calendar: gregorian
unlimited dimensions: 
current shape = (31,)
filling on, default _FillValue of -2147483647 used

In [23]:
lati = nc_file.variables["latitude"][:].filled(np.nan)
long = nc_file.variables["longitude"][:].filled(np.nan)
time = nc_file.variables["time"][:].filled(np.nan)
time = (pd.to_timedelta(time, unit="hours")+DATE_START).date
total_preci = nc_file.variables["tp"][:].filled(np.nan)

In [24]:
lat_3D, time_3D, long_3D = np.meshgrid(lati,time,long)

In [25]:
date = dt.date(2022,1,1)
good_date = np.where(time_3D[:,0,0] == date)[0][0]

In [26]:
time_3D[good_date,:,:].shape

(101, 151)

In [27]:
df = pd.DataFrame(
    {
    "lat":lat_3D[0,:,:].ravel(),
    "lon":long_3D[0,:,:].ravel(),
    "tp":total_preci[0,:,:].ravel(),
    }
)